In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
test = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip")
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
target = train[classes].values

In [ ]:
train.head()

In [ ]:
del train['id']
train.head()

In [ ]:
test.head()

In [ ]:
del test['id']
test.columns = ['comment_text']
test.head()

In [ ]:
train_sentences = train['comment_text']
test_sentences = test['comment_text']

In [ ]:
from keras.preprocessing.text import Tokenizer
max_tokens = 22000
tokens = Tokenizer(num_words = max_tokens, lower = True)
tokens.fit_on_texts(train_sentences)

In [ ]:
tokened_train = tokens.texts_to_sequences(train_sentences)
tokened_test = tokens.texts_to_sequences(test_sentences)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
max_len = 200
X_train = pad_sequences(tokened_train, maxlen = max_len)
X_test = pad_sequences(tokened_test, maxlen = max_len)

In [ ]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Bidirectional, Activation
model = keras.models.Sequential([keras.layers.Input(shape = (max_len)),
                                    keras.layers.Embedding(max_tokens, 128),
                                    keras.layers.Bidirectional(LSTM(56)),
                                    keras.layers.Dense(128, activation = 'relu'),
                                    keras.layers.Dropout(0.1),
                                    keras.layers.Dense(28, activation = 'relu'),
                                    keras.layers.Dropout(0.1),
                                    keras.layers.Dense(6, activation = 'sigmoid')])

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
model.fit(X_train, target, batch_size = 32, epochs = 2, validation_split = 0.1)

In [ ]:
model.save('model.h5')

In [ ]:
load_model = keras.models.load_model('./model.h5')

In [ ]:
y_test = load_model.predict(X_test)

In [ ]:
sample_submission = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')
sample_submission[classes] = y_test
sample_submission.to_csv('submission.csv', index = False)